## A "Greedy" change approach

In [1]:
def greedyChange(amount, denominations):
    # Goal is to produce the fewest coins to achieve
    # given target "amount"
    # Strategy: Give as many of the largest coin 
    # denomination that is less than amount. 
    solution = []
    for coin in denominations:
        i = amount // coin       # truncating integer divide
        solution.append(i)
        amount -= coin * i
    return solution

s1 = greedyChange(72, [25,10,5,1])
print(s1, sum(s1))
s2 = greedyChange(40, [25,10,5,1])
print(s2, sum(s2))
s3 = greedyChange(40, [25,20,10,5,1])
print(s3, sum(s3))

[2, 2, 0, 2] 6
[1, 1, 1, 0] 3
[1, 0, 1, 1, 0] 3


## Another Approach

In [2]:
def exhaustiveChange(amount, denominations):
    bestN = 100
    count = [0 for i in range(len(denominations))]
    while True:
        for i, coinValue in enumerate(denominations):
            count[i] += 1
            if (count[i]*coinValue < 100):
                break
            count[i] = 0
        n = sum(count)
        if n == 0:
            break
        value = sum([count[i]*denominations[i] for i in range(len(denominations))])
        if (value == amount):
            if (n < bestN):
                solution = [count[i] for i in range(len(denominations))]
                bestN = n
    return solution

%time print(exhaustiveChange(40,[25,20,10,5,1]))

[0, 2, 0, 0, 0]
CPU times: user 477 ms, sys: 3.2 ms, total: 480 ms
Wall time: 480 ms


## Correct, but costly
* Our algorithm now gets the right answer for every value 1..100
* It must, because it considers every possible answer<br>(that’s the good thing about brute force)
* There is a downside though

In [3]:
%time print(exhaustiveChange(40, [25,10,5,1]))
%time print(exhaustiveChange(40, [25,20,10,5,1]))
%time print(exhaustiveChange(40, [13,11,7,5,3,1]))

[1, 1, 1, 0]
CPU times: user 87.5 ms, sys: 1.97 ms, total: 89.5 ms
Wall time: 88.8 ms
[0, 2, 0, 0, 0]
CPU times: user 463 ms, sys: 1.88 ms, total: 465 ms
Wall time: 466 ms
[0, 3, 1, 0, 0, 0]
CPU times: user 1min 38s, sys: 505 ms, total: 1min 38s
Wall time: 1min 39s


## Other tricks?

A Branch-and-bound algorithm

In [4]:
def branchAndBoundChange(amount, denominations):
    bestN = amount
    count = [0 for i in range(len(denominations))]
    while True:
        for i, coinValue in enumerate(denominations):
            count[i] += 1
            if (count[i]*coinValue < amount):             # Set upper bound to amount rather than 100
                break
            count[i] = 0
        n = sum(count)
        if n == 0:
            break
        if (n > bestN):                                   # don't compute the amount if there are too many coins
            continue
        value = sum([count[i]*denominations[i] for i in range(len(denominations))])
        if (value == amount):
            if (n < bestN):
                solution = [count[i] for i in range(len(denominations))]
                bestN = n
    return solution

%time print(branchAndBoundChange(40, [13,11,7,5,3,1]))

[0, 3, 1, 0, 0, 0]
CPU times: user 188 ms, sys: 3.01 ms, total: 191 ms
Wall time: 191 ms


<h2>Other GREEDY Algorithms for Graphs</h2>
<h3>Algorithms for Minimum Spanning Trees</h3>


In [5]:
# Kruskal's and Prim's Minimum Spanning Tree (MST) algorithms.
# The program is for adjacency matrix representation of the graph
 
# Library for INT_MAX or could import math and use math.inf
import sys
 
 
class Graph():
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0 for column in range(vertices)]
                      for row in range(vertices)]
    
    
    def findEdges(self):
        """finds the edges in the undirected graph and puts them
        in a list in the form of [src, dest, weight]"""
        edges=[]
        for v in range(self.V):
            for a in range(self.V):
                if(self.graph[v][a] != 0 and [v,a,self.graph[v][a]] not in edges and [a,v,self.graph[a][v]]not in edges):
                    edges.append([v,a,self.graph[v][a] ])
        #print(edges)
        return edges
    
    # 
    def find(self, parent, i):
        """For Kruskals Algorithm helper function to find set of an element i"""
        if parent[i] != i:
          # Reassignment of node's parent to root node as
          # path compression requires
            parent[i] = self.find(parent, parent[i])
        return parent[i]
    

    def union(self, parent, rank, x, y):
        """A function that does union of two sets of x and y
        uses union by rank"""
        # Attach smaller rank tree under root of
        # high rank tree (Union by Rank)
        if rank[x] < rank[y]:
            parent[x] = y
        elif rank[x] > rank[y]:
            parent[y] = x
  
        # If ranks are same, then make one as root
        # and increment its rank by one
        else:
            parent[y] = x
            rank[x] += 1
  
 
    def KruskalMST(self):
        """The main function to construct MST using Kruskal's
        returns the list of edges in the MST"""
        result = []  # This will store the resultant MST
  
        # An index variable, used for sorted edges
        i = 0
  
        # An index variable, used for result[]
        e = 0
  
        # Step 1:  Sort all the edges in
        # non-decreasing order of their
        # weight.  If we are not allowed to change the
        # given graph, we can create a copy of graph
        sortedEdges = sorted(self.findEdges(),
                            key=lambda item: item[2])
  
        #print(sortedEdges)
        parent = []
        rank = []
  
        # Create V subsets with single elements
        for node in range(self.V):
            parent.append(node)
            rank.append(0)
  
        # Number of edges to be taken is less than to V-1
        while e < self.V - 1:
  
            # Step 2: Pick the smallest edge and increment
            # the index for next iteration
            u, v, w = sortedEdges[i]
            i = i + 1
            x = self.find(parent, u)
            y = self.find(parent, v)
  
            # If including this edge doesn't
            # cause cycle, then include it in result
            # and increment the index of result
            # for next edge
            if x != y:
                e = e + 1
                result.append([u, v, w])
                self.union(parent, rank, x, y)
            # Else discard the edge
        self.printMST(result,"Kruskal's")
        return result
    
    
    def printMST(self, result, technique):
        """prints the edges in the MST """
        minimumCost = 0
        print(technique)
        print("Edge \tWeight")
        for u, v, weight in result:
            minimumCost += weight
            print(f"{u} - {v}\t{weight}")
            #print("%d -- %d == %d" % (u, v, weight))
        print(f"Minimum Spanning Tree: {minimumCost}")
    
    
    def minKey(self, key, mstSet):
        """A helper function for Prim's  and Dijkstras to find the vertex with
        minimum distance value, from the set of vertices
        not yet included in shortest path tree"""
        # Initialize min value to be positive inf
        min = sys.maxsize
 
        for v in range(self.V):
            if key[v] < min and mstSet[v] == False:
                min = key[v]
                min_index = v
 
        return min_index
 
    def primMST(self):
        """Function to construct and print MST for a graph
        represented using adjacency matrix representation"""
        result= []
        # Key values used to pick minimum weight edge in cut
        key = [sys.maxsize] * self.V
        parent = [None] * self.V  # Array to store constructed MST
        # Make key 0 so that this vertex is picked as first vertex
        key[0] = 0
        mstSet = [False] * self.V
 
        parent[0] = -1  # First node is always the root of
 
        for cout in range(self.V):
 
            # Pick the minimum distance vertex from
            # the set of vertices not yet processed.
            # u is always equal to src in first iteration
            u = self.minKey(key, mstSet)
 
            # Put the minimum distance vertex in
            # the shortest path tree
            mstSet[u] = True
 
            # Update dist value of the adjacent vertices
            # of the picked vertex only if the current
            # distance is greater than new distance and
            # the vertex in not in the shortest path tree
            for v in range(self.V):
 
                # graph[u][v] is non zero only for adjacent vertices of m
                # mstSet[v] is false for vertices not yet included in MST
                # Update the key only if graph[u][v] is smaller than key[v]
                if self.graph[u][v] > 0 and mstSet[v] == False \
                and key[v] > self.graph[u][v]:
                    key[v] = self.graph[u][v]
                    parent[v] = u
                    
        result = self.makePrimSolution(parent)
        self.printMST(result,"Prim's")
        return result
    
    def makePrimSolution(self, parentList):
        """makes the list based on the parent list given from Prim's"""
        result=[]
        for i in range(1, self.V):
            result.append([parentList[i], i, self.graph[i][parentList[i]]])
        return result
    
    def renderGraph(self):
        """ Outputs a version of the graph that can be rendered
        using graphviz tools (http://www.graphviz.org/)."""
        edges=self.findEdges()#find edges
        result = ''
        result += 'graph {\n'
        result += '   graph [nodesep=2, size="10,10"];\n'
        for v in range(self.V):
            result += '    N%d [shape="box", style="rounded", label="%s"];\n' % (v, v)
        #print(result)
        for e in edges:
            src, dst, label = e
            result += '    N%d -- N%d' % (src, dst)
            result += ' [label="%s"]' % (label)              
            result += ';\n'                
        result += '    overlap=false;\n'
        result += '}\n'
        return result
    
    
    def renderGraphMST(self, mstEdges):
        """ Outputs a version of the graph withs MST in red that can be rendered
        using graphviz tools (http://www.graphviz.org/)."""
        edges=self.findEdges()#find edges
        result = ''
        result += 'graph {\n'
        result += '   graph [nodesep=2, size="10,10"];\n'
        for v in range(self.V):
            result += '    N%d [shape="box", style="rounded", label="%s"];\n' % (v, v)
        #print(result)

        for e in edges:
            src, dst, label = e
            if e in mstEdges or [dst, src, label]in mstEdges:
                result += '    N%d -- N%d [color=red,penwidth=3.0]' % (src, dst)
            else:
                result += '    N%d -- N%d' % (src, dst)
            result += ' [label="%s"]' % (label)              
            result += ';\n'                
        result += '    overlap=true;\n'
        result += '}\n'
        return result        
    
    
    
    
    def printDijkstrasSolution(self, dist):
        print("Vertex \t Distance from Source")
        for node in range(self.V):
            print(node, "\t\t", dist[node])
    
    # Function that implements Dijkstra's single source
    # shortest path algorithm for a graph represented
    # using adjacency matrix representation
    def dijkstra(self, src):
 
        dist = [1e7] * self.V
        dist[src] = 0
        sptSet = [False] * self.V
 
        for cout in range(self.V):
 
            # Pick the minimum distance vertex from
            # the set of vertices not yet processed.
            # u is always equal to src in first iteration
            u = self.minKey(dist, sptSet)
 
            # Put the minimum distance vertex in the
            # shortest path tree
            sptSet[u] = True
 
            # Update dist value of the adjacent vertices
            # of the picked vertex only if the current
            # distance is greater than new distance and
            # the vertex in not in the shortest path tree
            for v in range(self.V):
                if (self.graph[u][v] > 0 and
                   sptSet[v] == False and
                   dist[v] > dist[u] + self.graph[u][v]):
                    dist[v] = dist[u] + self.graph[u][v]
 
        self.printDijkstrasSolution(dist)

In [24]:
matrix = [[0, 4, 0, 3, 5], [4, 0, 2, 0, 0], [0, 2, 0, 1, 0], [3, 0, 1, 0, 0], [5, 0, 0, 0, 0]]
exampleGraph = Graph(len(matrix))
exampleGraph.graph = matrix

rendering = exampleGraph.renderGraph()
print(rendering)
with open("ExampleGraph.dot", 'w') as fp:
    fp.write(rendering)

graph {
   graph [nodesep=2, size="10,10"];
    N0 [shape="box", style="rounded", label="0"];
    N1 [shape="box", style="rounded", label="1"];
    N2 [shape="box", style="rounded", label="2"];
    N3 [shape="box", style="rounded", label="3"];
    N4 [shape="box", style="rounded", label="4"];
    N0 -- N1 [label="4"];
    N0 -- N3 [label="3"];
    N0 -- N4 [label="5"];
    N1 -- N2 [label="2"];
    N2 -- N3 [label="1"];
    overlap=false;
}



In [7]:
# The following command assumes that graphviz is installed.
# See https://www.graphviz.org/
!circo -Goverlap=scale -Tpng ExampleGraph.dot -o ExampleGraph.png

zsh:1: command not found: circo


In [8]:
from IPython.display import Image
Image('ExampleGraph.png')

FileNotFoundError: No such file or directory: 'ExampleGraph.png'

FileNotFoundError: No such file or directory: 'ExampleGraph.png'

<IPython.core.display.Image object>

In [9]:
kruskalEdges = exampleGraph.KruskalMST()

Kruskal's
Edge 	Weight
2 - 3	1
1 - 2	2
0 - 3	3
0 - 4	5
Minimum Spanning Tree: 11


In [10]:
mstRender = exampleGraph.renderGraphMST(kruskalEdges)
print(mstRender)
with open("KruskalMST.dot", 'w') as fp:
    fp.write(mstRender)

graph {
   graph [nodesep=2, size="10,10"];
    N0 [shape="box", style="rounded", label="0"];
    N1 [shape="box", style="rounded", label="1"];
    N2 [shape="box", style="rounded", label="2"];
    N3 [shape="box", style="rounded", label="3"];
    N4 [shape="box", style="rounded", label="4"];
    N0 -- N1 [label="4"];
    N0 -- N3 [color=red,penwidth=3.0] [label="3"];
    N0 -- N4 [color=red,penwidth=3.0] [label="5"];
    N1 -- N2 [color=red,penwidth=3.0] [label="2"];
    N2 -- N3 [color=red,penwidth=3.0] [label="1"];
    overlap=true;
}



In [11]:
# The following command assumes that graphviz is installed.
# See https://www.graphviz.org/
!circo -Goverlap=scale -Tpng KruskalMST.dot -o KruskalMST.png

zsh:1: command not found: circo


In [12]:
from IPython.display import Image
Image('KruskalMST.png')

FileNotFoundError: No such file or directory: 'KruskalMST.png'

FileNotFoundError: No such file or directory: 'KruskalMST.png'

<IPython.core.display.Image object>

In [13]:
primEdges = exampleGraph.primMST()

Prim's
Edge 	Weight
2 - 1	2
3 - 2	1
0 - 3	3
0 - 4	5
Minimum Spanning Tree: 11


In [14]:
mstRender = exampleGraph.renderGraphMST(primEdges)
print(mstRender)
with open("PrimMST.dot", 'w') as fp:
    fp.write(mstRender)

graph {
   graph [nodesep=2, size="10,10"];
    N0 [shape="box", style="rounded", label="0"];
    N1 [shape="box", style="rounded", label="1"];
    N2 [shape="box", style="rounded", label="2"];
    N3 [shape="box", style="rounded", label="3"];
    N4 [shape="box", style="rounded", label="4"];
    N0 -- N1 [label="4"];
    N0 -- N3 [color=red,penwidth=3.0] [label="3"];
    N0 -- N4 [color=red,penwidth=3.0] [label="5"];
    N1 -- N2 [color=red,penwidth=3.0] [label="2"];
    N2 -- N3 [color=red,penwidth=3.0] [label="1"];
    overlap=true;
}



In [15]:
# The following command assumes that graphviz is installed.
# See https://www.graphviz.org/
!circo -Goverlap=scale -Tpng PrimMST.dot -o PrimMST.png

zsh:1: command not found: circo


In [16]:
from IPython.display import Image
Image('PrimMST.png')

FileNotFoundError: No such file or directory: 'PrimMST.png'

FileNotFoundError: No such file or directory: 'PrimMST.png'

<IPython.core.display.Image object>

In [17]:
matrix = [[0, 2, 0, 6, 0], [2, 0, 3, 8, 5], [0, 3, 0, 0, 7],[6, 8, 0, 0, 9],  [0, 5, 7, 9, 0]]
exampleGraph = Graph(len(matrix))
exampleGraph.graph = matrix
kruskalEdges = exampleGraph.KruskalMST()
primEdges = exampleGraph.primMST()
 
mstRender = exampleGraph.renderGraphMST(kruskalEdges)
print(mstRender)
with open("AnotherKruskalMST.dot", 'w') as fp:
    fp.write(mstRender)

    
mstRender = exampleGraph.renderGraphMST(primEdges)
print(mstRender)
with open("AnotherPrimMST.dot", 'w') as fp:
    fp.write(mstRender)

Kruskal's
Edge 	Weight
0 - 1	2
1 - 2	3
1 - 4	5
0 - 3	6
Minimum Spanning Tree: 16
Prim's
Edge 	Weight
0 - 1	2
1 - 2	3
0 - 3	6
1 - 4	5
Minimum Spanning Tree: 16
graph {
   graph [nodesep=2, size="10,10"];
    N0 [shape="box", style="rounded", label="0"];
    N1 [shape="box", style="rounded", label="1"];
    N2 [shape="box", style="rounded", label="2"];
    N3 [shape="box", style="rounded", label="3"];
    N4 [shape="box", style="rounded", label="4"];
    N0 -- N1 [color=red,penwidth=3.0] [label="2"];
    N0 -- N3 [color=red,penwidth=3.0] [label="6"];
    N1 -- N2 [color=red,penwidth=3.0] [label="3"];
    N1 -- N3 [label="8"];
    N1 -- N4 [color=red,penwidth=3.0] [label="5"];
    N2 -- N4 [label="7"];
    N3 -- N4 [label="9"];
    overlap=true;
}

graph {
   graph [nodesep=2, size="10,10"];
    N0 [shape="box", style="rounded", label="0"];
    N1 [shape="box", style="rounded", label="1"];
    N2 [shape="box", style="rounded", label="2"];
    N3 [shape="box", style="rounded", label="3"];


In [18]:
# The following command assumes that graphviz is installed.
# See https://www.graphviz.org/
!circo -Goverlap=scale -Tpng AnotherKruskalMST.dot -o AnotherKruskalMST.png

zsh:1: command not found: circo


In [19]:
from IPython.display import Image
Image('AnotherKruskalMST.png')

FileNotFoundError: No such file or directory: 'AnotherKruskalMST.png'

FileNotFoundError: No such file or directory: 'AnotherKruskalMST.png'

<IPython.core.display.Image object>

In [20]:
# The following command assumes that graphviz is installed.
# See https://www.graphviz.org/
!circo -Goverlap=scale -Tpng AnotherPrimMST.dot -o AnotherPrimMST.png

zsh:1: command not found: circo


In [21]:
from IPython.display import Image
Image('AnotherPrimMST.png')

FileNotFoundError: No such file or directory: 'AnotherPrimMST.png'

FileNotFoundError: No such file or directory: 'AnotherPrimMST.png'

<IPython.core.display.Image object>

In [22]:
matrix2 = [[0, 4, 0, 0, 0, 0, 0, 8, 0],
           [4, 0, 8, 0, 0, 0, 0, 11, 0], 
           [0, 8, 0, 7, 0, 4, 0, 0, 2],
           [0, 0, 7, 0, 9, 14, 0, 0, 0],
           [0, 0, 0, 9, 0, 10, 0, 0, 0],
           [0, 0, 4, 14, 10, 0, 2, 0, 0], 
           [0, 0, 0, 0, 0, 2, 0, 1, 6],
           [8, 11, 0, 0, 0, 0, 1, 0, 7], 
           [0, 0, 2, 0, 0, 0, 6, 7, 0]]
exampleGraphD = Graph(len(matrix2))
exampleGraphD.graph = matrix2
exampleGraphD.dijkstra(0)

Vertex 	 Distance from Source
0 		 0
1 		 4
2 		 12
3 		 19
4 		 21
5 		 11
6 		 9
7 		 8
8 		 14


In [23]:
for v in range(len(exampleGraphD.graph)):
    print(f"Distances from vertex {v}")
    exampleGraphD.dijkstra(v)

Distances from vertex 0
Vertex 	 Distance from Source
0 		 0
1 		 4
2 		 12
3 		 19
4 		 21
5 		 11
6 		 9
7 		 8
8 		 14
Distances from vertex 1
Vertex 	 Distance from Source
0 		 4
1 		 0
2 		 8
3 		 15
4 		 22
5 		 12
6 		 12
7 		 11
8 		 10
Distances from vertex 2
Vertex 	 Distance from Source
0 		 12
1 		 8
2 		 0
3 		 7
4 		 14
5 		 4
6 		 6
7 		 7
8 		 2
Distances from vertex 3
Vertex 	 Distance from Source
0 		 19
1 		 15
2 		 7
3 		 0
4 		 9
5 		 11
6 		 13
7 		 14
8 		 9
Distances from vertex 4
Vertex 	 Distance from Source
0 		 21
1 		 22
2 		 14
3 		 9
4 		 0
5 		 10
6 		 12
7 		 13
8 		 16
Distances from vertex 5
Vertex 	 Distance from Source
0 		 11
1 		 12
2 		 4
3 		 11
4 		 10
5 		 0
6 		 2
7 		 3
8 		 6
Distances from vertex 6
Vertex 	 Distance from Source
0 		 9
1 		 12
2 		 6
3 		 13
4 		 12
5 		 2
6 		 0
7 		 1
8 		 6
Distances from vertex 7
Vertex 	 Distance from Source
0 		 8
1 		 11
2 		 7
3 		 14
4 		 13
5 		 3
6 		 1
7 		 0
8 		 7
Distances from vertex 8
Verte